In [1]:
import cv2
import sys
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
def preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load image
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            # Resize image to 28x28
            #image = cv2.resize(image, (28, 28))
            
            # Convert image to numpy array
            image = np.array(image)
            
            # Normalize pixel values to range [0, 1]
            image = image / 255.0
            
            # Append image and label to lists
            images.append(image)
            labels.append(label)
    
    return images, labels

# Paths to positive (1) and negative (0) class folders
positive_folder = "archive/datasets/train_folder/1"
negative_folder = "archive/datasets/train_folder/0"

# Load and preprocess images from positive class folder
positive_images, positive_labels = preprocess_images(positive_folder, 1)

# Load and preprocess images from negative class folder
negative_images, negative_labels = preprocess_images(negative_folder, 0)

# Concatenate positive and negative images and labels
images = np.concatenate((positive_images, negative_images), axis=0)
labels = np.concatenate((positive_labels, negative_labels), axis=0)

# Shuffle images and labels
random_indices = np.random.permutation(len(images))
images = images[random_indices]
labels = labels[random_indices]

# Reshape images to fit LeNet input shape (add channel dimension)
images = np.expand_dims(images, axis=-1)

# Print shape information
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (2800, 64, 64, 1)
Labels shape: (2800,)


In [3]:
images_train = images
labels_train = labels

In [4]:
def preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load image
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            # Resize image to 28x28
            #image = cv2.resize(image, (28, 28))
            
            # Convert image to numpy array
            image = np.array(image)
            
            # Normalize pixel values to range [0, 1]
            image = image / 255.0
            
            # Append image and label to lists
            images.append(image)
            labels.append(label)
    
    return images, labels

# Paths to positive (1) and negative (0) class folders
positive_folder = "archive/datasets/test_folder/1"
negative_folder = "archive/datasets/test_folder/0"

# Load and preprocess images from positive class folder
positive_images, positive_labels = preprocess_images(positive_folder, 1)

# Load and preprocess images from negative class folder
negative_images, negative_labels = preprocess_images(negative_folder, 0)

# Concatenate positive and negative images and labels
images = np.concatenate((positive_images, negative_images), axis=0)
labels = np.concatenate((positive_labels, negative_labels), axis=0)

# Shuffle images and labels
random_indices = np.random.permutation(len(images))
images = images[random_indices]
labels = labels[random_indices]

# Reshape images to fit LeNet input shape (add channel dimension)
images = np.expand_dims(images, axis=-1)

# Print shape information
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (1200, 64, 64, 1)
Labels shape: (1200,)


In [5]:
images_test = images
labels_test = labels

In [6]:
model = Sequential([
    Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Step 3: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Train the model on the training data
model.fit(images_train, labels_train, batch_size=32, epochs=10, validation_split=0.1)

# Step 5: Evaluate the trained model on the testing data
loss, accuracy = model.evaluate(images_test, labels_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/10
79/79 [==============================] - 2s 18ms/step - loss: 0.6573 - accuracy: 0.6103 - val_loss: 0.6058 - val_accuracy: 0.6750
Epoch 2/10
79/79 [==============================] - 1s 16ms/step - loss: 0.4882 - accuracy: 0.7710 - val_loss: 0.4152 - val_accuracy: 0.8286
Epoch 3/10
79/79 [==============================] - 1s 14ms/step - loss: 0.3620 - accuracy: 0.8440 - val_loss: 0.3657 - val_accuracy: 0.8500
Epoch 4/10
79/79 [==============================] - 1s 14ms/step - loss: 0.3386 - accuracy: 0.8560 - val_loss: 0.3593 - val_accuracy: 0.8536
Epoch 5/10
79/79 [==============================] - 1s 15ms/step - loss: 0.2978 - accuracy: 0.8671 - val_loss: 0.3930 - val_accuracy: 0.8536
Epoch 6/10
79/79 [==============================] - 1s 15ms/step - loss: 0.2692 - accuracy: 0.8893 - val_loss: 0.3933 - val_accuracy: 0.8357
Epoch 7/10
79/79 [==============================] - 1s 15ms/step - loss: 0.2440 - accuracy: 0.8960 - val_loss: 0.4895 - val_accuracy: 0.8357
Epoch 8/10
79

In [7]:
cascPath = sys.argv[1]
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        #cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # Extract the face region from the frame
        face_region = gray[y:y+h, x:x+w]
        # Resize the face region to match the input size expected by the model
        face_region = cv2.resize(face_region, (64, 64))
        # Normalize the pixel values to [0, 1]
        face_region = face_region / 255.0
        # Expand dimensions to match the input shape expected by the model
        face_region = np.expand_dims(face_region, axis=0)
        face_region = np.expand_dims(face_region, axis=-1)

        # Make prediction using the model
        prediction = model.predict(face_region)
        # Assuming binary classification, if prediction is closer to 1, label as "Positive", otherwise "Negative"
        label = "Positive" if prediction > 0.5 else "Negative"

        # Draw a rectangle around the face
        if label == "Positive":
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # Display the predicted label on the frame
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        # Display the predicted label on the frame
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # use model to make prediction on faces
    

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 77ms/step


2024-03-25 18:18:57.159 Python[3447:196809] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


1/1 [==============================] - 0s 12ms/step


: 